In [1]:
import pandas as pd
import folium

In [3]:
stations = pd.read_csv("ev_stations.csv")
zips = pd.read_csv("uszips.csv")
acs = pd.read_csv("ACSST5Y2023.S1903-Data.csv", skiprows=1)

/tmp/ipython-input-3-3631600075.py:1: DtypeWarning: Columns (6,20,31,46,69) have mixed types. Specify dtype option on import or set low_memory=False.
  stations = pd.read_csv("ev_stations.csv")
/tmp/ipython-input-3-3631600075.py:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  zips = pd.read_csv("uszips.csv")


In [4]:
stations['ZIP'] = stations['ZIP'].astype(str).str[:5]
stations_ca = stations[stations['State'] == 'CA']
charger_count = stations_ca.groupby('ZIP').size().reset_index(name='charger_count')

In [5]:
zips = zips.rename(columns={'zip': 'ZIP', 'lat': 'lat', 'lng': 'lon'})
zips['ZIP'] = zips['ZIP'].astype(str)

In [6]:
zips_ca = zips[zips['state_id'] == 'CA'][['ZIP', 'population', 'lat', 'lon']]
merged = pd.merge(charger_count, zips_ca, on='ZIP', how='left')
merged['chargers_per_10k'] = (merged['charger_count'] / merged['population']) * 10000

In [7]:
income_col = "Estimate!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households"
acs_income = acs[['Geographic Area Name', income_col]].copy()
acs_income.columns = ['ZIP_Name', 'Median_Income']
acs_income['ZIP'] = acs_income['ZIP_Name'].str.extract(r'ZCTA5 (\d{5})')
acs_income['Median_Income'] = pd.to_numeric(acs_income['Median_Income'], errors='coerce')
acs_income = acs_income.dropna(subset=['ZIP', 'Median_Income'])
acs_income['ZIP'] = acs_income['ZIP'].astype(str)

In [8]:
merged = pd.merge(merged, acs_income[['ZIP', 'Median_Income']], on='ZIP', how='left')
merged['low_income'] = merged['Median_Income'] < 45000

In [9]:
merged['underserved'] = (merged['charger_count'] < 2) & (merged['low_income'])
underserved_zips = merged[merged['underserved']]

In [10]:
m = folium.Map(location=[34.05, -118.25], zoom_start=10)

for _, row in underserved_zips.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"ZIP: {row['ZIP']}, Income: ${int(row['Median_Income'])}"
    ).add_to(m)

m
